In [41]:
%pip install app_store_scraper

Note: you may need to restart the kernel to use updated packages.


In [42]:
import pandas as pd
import numpy as np
import json
import csv
import time
from app_store_scraper import AppStore


with open('DATA/app_list.json', mode='r', encoding='utf-8') as f:
    app_data = json.load(f) 

2023-01-28 07:26:19,358 [INFO] Base - Initialised: AppStore('us', 'chime', 836215269)
2023-01-28 07:26:19,359 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/chime/id836215269


In [ ]:
import csv
import time

import pandas as pd
import numpy as np
import json

from app_store_scraper import AppStore
#scrap data from Appstore
app_data = dict(zip(app_names, app_ids))

with open('app_data.csv', mode='w') as file:
    writer = csv.writer(file)
    writer.writerow(['App Name', 'App ID'])
    for app_name, app_id in app_data.items():
        writer.writerow([app_name, app_id])
        
reviews_per_iteration = 2000
for app_name, app_id in app_data.items():
    while True:
        try:
            app_store = AppStore(country='us', app_name=app_name, app_id = app_id)
            reviews = app_store.review(how_many=reviews_per_iteration)
            app_reviews_df= pd.DataFrame(np.array(app_store.reviews),columns=['review'])
            app_reviews_df= app_reviews_df.join(pd.DataFrame(app_reviews_df.pop('review').tolist()))
            app_reviews_df.to_csv(f'{app_name}_reviews.csv', index=False)
            time.sleep(120)
            break
        except Exception as e:
            print(f"{app_name} not found, retrying after 120 seconds")
            print(e)
            time.sleep(60)


2023-01-27 20:58:50,026 [INFO] Base - Initialised: AppStore('us', 'acorns', 883324671)
2023-01-27 20:58:50,028 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/acorns/id883324671
2023-01-27 20:58:55,087 [INFO] Base - [id:883324671] Fetched 340 reviews (340 fetched in total)
2023-01-27 20:59:00,277 [INFO] Base - [id:883324671] Fetched 1080 reviews (1080 fetched in total)
2023-01-27 20:59:05,970 [INFO] Base - [id:883324671] Fetched 1460 reviews (1460 fetched in total)
2023-01-27 20:59:11,451 [INFO] Base - [id:883324671] Fetched 1820 reviews (1820 fetched in total)
2023-01-27 20:59:14,392 [INFO] Base - [id:883324671] Fetched 2000 reviews (2000 fetched in total)
2023-01-27 21:01:15,137 [INFO] Base - Initialised: AppStore('us', 'betterment-', 393156562)
2023-01-27 21:01:15,146 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/betterment-/id393156562
2023-01-27 21:01:20,179 [INFO] Base - [id:393156562] Fetched 720 reviews (720 fetched in total)


In [1]:
import pandas as pd
import os

# Set the path to the folder containing the excel files
path = '/Users/toyosibamidele/Desktop/fw/apps'

# Initialize an empty list to store the dataframes
df_list = []

# Iterate over the excel files in the folder
for file in os.listdir(path):
    if file.endswith(".csv"):
        # Check if the file is empty
        if os.path.getsize(os.path.join(path, file)) > 1:
            # Read the excel file into a dataframe
            df = pd.read_csv(os.path.join(path, file), usecols=['date','review','rating','isEdited','title','userName'])
            # Use os.path.splitext() to get the file name without the extension
            file_name = os.path.splitext(file)[0]
            # remove _reviews
            file_name = file_name.replace("_reviews", "")
            # Add a column with the file name
            df['file_name'] = file_name
            # Append the dataframe to the list
            df_list.append(df)

# Concatenate the dataframes together
merged_df = pd.concat(df_list, ignore_index=True)


In [2]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50218 entries, 0 to 50217
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       50218 non-null  object
 1   review     50217 non-null  object
 2   rating     50218 non-null  int64 
 3   isEdited   50218 non-null  bool  
 4   userName   50218 non-null  object
 5   title      50218 non-null  object
 6   file_name  50218 non-null  object
dtypes: bool(1), int64(1), object(5)
memory usage: 2.3+ MB


In [3]:
import pandas as pd

# Load the Excel file into a DataFrame
reviews_df = merged_df

In [4]:
#%pip install nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Initialize the stemmer
stemmer = PorterStemmer()

# Remove stopwords
stop_words = set(stopwords.words("english"))
reviews_df["review_text"] = reviews_df["review"].apply(lambda x: " ".join([word for word in word_tokenize(x) if word not in stop_words]) if pd.notnull(x) else '')
reviews_df["review_text"] = reviews_df["review"].apply(lambda x: " ".join([stemmer.stem(word) for word in word_tokenize(x)]) if pd.notnull(x) else '')
reviews_df["title_text"] = reviews_df["title"].apply(lambda x: " ".join([word for word in word_tokenize(x) if word not in stop_words]) if pd.notnull(x) else '')
reviews_df["title_text"] = reviews_df["title"].apply(lambda x: " ".join([stemmer.stem(word) for word in word_tokenize(x)]) if pd.notnull(x) else '')



In [5]:
reviews_df.head()

,date,review,rating,isEdited,userName,title,file_name,review_text,title_text
0,2013-12-04 19:55:58,Makes banking easy,5,False,JC1973a,Great app!,THE BANCORP BANK,make bank easi,great app !
1,2014-04-19 08:33:39,Was very excited when I found this app. I have...,1,False,KateSpivey,Disappointed,THE BANCORP BANK,wa veri excit when i found thi app . i have an...,disappoint
2,2014-06-16 23:00:35,I am very disappointed that the app doesn't wo...,1,False,ericgibson74,Disappointed,THE BANCORP BANK,i am veri disappoint that the app doe n't work...,disappoint
3,2014-02-26 14:43:05,App access is unavailable for people who only ...,1,False,Soxhound,Not Usable,THE BANCORP BANK,app access is unavail for peopl who onli have ...,not usabl
4,2014-03-13 03:13:35,Doesn't work for an HSA! You need to have othe...,1,False,newlife503,JUNK!,THE BANCORP BANK,doe n't work for an hsa ! you need to have oth...,junk !


In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Fine-tune a BERT model on the reviews
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
from transformers import pipeline

# Initialize the sentiment classifier pipeline
sentiment_classifier = pipeline("sentiment-analysis", truncation=True, max_length=512)

# Get the sentiment predictions for the 'review_text' column
reviews_df["sentiment_review"] = reviews_df["review_text"].apply(lambda x: sentiment_classifier(x)[0]['label'])

# Get the sentiment predictions for the 'title_text' column
reviews_df["sentiment_title"] = reviews_df["title_text"].apply(lambda x: sentiment_classifier(x)[0]['label'])



No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [18]:
from transformers import pipeline

# Initialize the sentiment classifier pipeline
sentiment_classifier = pipeline("sentiment-analysis", truncation=True, max_length=512)

# Get the sentiment predictions for the 'review_text' column
reviews_df["sentiment_review_score"] = reviews_df["review_text"].apply(lambda x: sentiment_classifier(x)[0]['score'])

# Get the sentiment predictions for the 'title_text' column
reviews_df["sentiment_title_score"] = reviews_df["title_text"].apply(lambda x: sentiment_classifier(x)[0]['score'])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [20]:
reviews_df.to_csv('reviews_data_01_28.csv', index = False)

In [21]:
reviews_df.head()

,date,review,rating,isEdited,userName,title,file_name,review_text,title_text,sentiment_review,sentiment_title,sentiment_review_score,sentiment_title_score
0,2013-12-04 19:55:58,Makes banking easy,5,False,JC1973a,Great app!,THE BANCORP BANK,make bank easi,great app !,POSITIVE,POSITIVE,0.992134,0.999846
1,2014-04-19 08:33:39,Was very excited when I found this app. I have...,1,False,KateSpivey,Disappointed,THE BANCORP BANK,wa veri excit when i found thi app . i have an...,disappoint,NEGATIVE,NEGATIVE,0.998448,0.999520
2,2014-06-16 23:00:35,I am very disappointed that the app doesn't wo...,1,False,ericgibson74,Disappointed,THE BANCORP BANK,i am veri disappoint that the app doe n't work...,disappoint,NEGATIVE,NEGATIVE,0.997736,0.999520
3,2014-02-26 14:43:05,App access is unavailable for people who only ...,1,False,Soxhound,Not Usable,THE BANCORP BANK,app access is unavail for peopl who onli have ...,not usabl,NEGATIVE,NEGATIVE,0.964456,0.991257
4,2014-03-13 03:13:35,Doesn't work for an HSA! You need to have othe...,1,False,newlife503,JUNK!,THE BANCORP BANK,doe n't work for an hsa ! you need to have oth...,junk !,NEGATIVE,NEGATIVE,0.999334,0.999786


In [ ]:
import nltk

# Tokenize the reviews
reviews_df["review_tokens"] = reviews_df["review_text"].apply(lambda x: nltk.word_tokenize(x))

# Create a list of negative reviews
negative_reviews = reviews_df[reviews_df["sentiment"] == "negative"]["review_tokens"]

# Create a frequency distribution of the words in the negative reviews
fdist = nltk.FreqDist(word for review in negative_reviews for word in review)


#%pip install nltk
import nltk
from nltk.corpus import stopwords

# Get the list of stop words
stop_words = set(stopwords.words("english"))

# Initialize an empty dictionary to store the lists of negative words
negative_words_by_file = {}

# Tokenize the reviews
reviews_df["review_tokens"] = reviews_df["review"].apply(lambda x: nltk.word_tokenize(x))

# Loop through the unique file names
for file_name in reviews_df['file_name'].unique():
    # Get the rows of the dataframe that have the current file name
    file_reviews_df = reviews_df[reviews_df['file_name'] == file_name]
    # Get the negative reviews for the current file
    negative_reviews = file_reviews_df[file_reviews_df["sentiment_review"] == "NEGATIVE"]["review_tokens"]
    # Create a frequency distribution of the words in the negative reviews, removing stop words
    fdist = nltk.FreqDist(word for review in negative_reviews for word in review if word not in stop_words)
    # Get the top 20 most frequent words
    negative_words = [word for word, freq in fdist.most_common(20)]
    # Add the list of negative words to the dictionary with the current file name as the key
    negative_words_by_file[file_name] = negative_words


# Import the necessary modules
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Initialize an empty dictionary to store the lists of negative words
negative_words_by_file = {}

# Tokenize the reviews and remove punctuation
reviews_df["review_tokens"] = reviews_df["review"].apply(lambda x: word_tokenize(x.translate(str.maketrans('', '', string.punctuation))))
reviews_df["review_tokens"] = reviews_df["review_tokens"].apply(lambda x: [word.lower() for word in x])

# Loop through the unique file names
for file_name in reviews_df['file_name'].unique():
    # Get the rows of the dataframe that have the current file name
    file_reviews_df = reviews_df[reviews_df['file_name'] == file_name]
    # Get the negative reviews for the current file
    negative_reviews = file_reviews_df[file_reviews_df["sentiment_review"] == "NEGATIVE"]["review_tokens"]
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    negative_reviews = [[word for word in review if word not in stop_words] for review in negative_reviews]
    # Create a frequency distribution of the words in the negative reviews
    fdist = nltk.FreqDist(word for review in negative_reviews for word in review)
    # Get the top 20 most frequent words
    negative_words = [word for word, freq in fdist.most_common(20)]
    # Add the list of negative words to the dictionary with the current file name as the key
    negative_words_by_file[file_name] = negative_words


In [ ]:
# Initialize an empty dictionary to store results
results = {}

# Get a list of unique file names
file_names = reviews_df['filename'].unique()

# Iterate through the file names
for file_name in file_names:
    # Filter the dataframe to only include rows with the current file name
    file_df = reviews_df[reviews_df['filename'] == file_name]
    
    # Create a list of negative reviews for the current file name
    negative_reviews = file_df[file_df["sentiment"] == "negative"]["review_tokens"]
    
    # Create a frequency distribution of the words in the negative reviews
    fdist = nltk.FreqDist(word for review in negative_reviews for word in review)
    
    # Get the most common phrase
    most_common_phrase = fdist.most_common(1)
    
    # Add the file name and most common phrase to the results dictionary
    results[file_name] = most_common_phrase


In [ ]:
pain_points = fdist.most_common(20)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a list of negative review texts
negative_reviews_text = reviews_df[reviews_df["sentiment"] == "negative"]["review_text"]

# Create the TF-IDF matrix
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(negative_reviews_text)

# Get the feature names
feature_names = tfidf.get_feature_names()

# Get the tf-idf score of each word
tfidf_scores = dict(zip(feature_names, tfidf.idf_))

# Sort the tf-idf scores to identify the most important words
sorted_tfidf_scores = sorted(tfidf_scores.items(), key=lambda x: x[1], reverse=True)


In [ ]:
import matplotlib.pyplot as plt

# Count the number of reviews for each sentiment
sentiment_counts = reviews_df["sentiment"].value_counts()

# Create the bar chart
sentiment_counts.plot(kind="bar", xlabel="Sentiment", ylabel="Number of Reviews")
plt.show()


In [ ]:
from wordcloud import WordCloud

# Create a list of words from the negative reviews
words = [word for review in negative_reviews for word in review]

# Create the word cloud
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(" ".join(words))

plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show()


In [ ]:
# Get the top 20 pain points
top_20_pain_points = fdist.most_common(20)

# Extract the words and frequencies
words = [word[0] for word in top_20_pain_points]
freqs = [word[1] for word in top_20_pain_points]

# Create the bar chart
plt.bar(words, freqs)
plt.xlabel("Words")
plt.ylabel("Frequency")
plt.xticks(rotation=90)
plt.show()
